# Scrape scheme details using the urls

In [ ]:
!pip install playwright
!playwright install

## Scraping scheme info

In [7]:
import csv
import asyncio
import html
import os
from tqdm import tqdm
from unidecode import unidecode
from playwright.async_api import async_playwright

In [2]:
# Output file
output_file = 'scheme_data.csv'

In [3]:
# Header
columns = ['Scheme Name', 'URL', 'State', 'Ministry', 'Details', 'Benefits', 'Eligibility',
           'Application Process', 'Documents Required', 'Search Keywords', 'Features']

In [4]:
# Create CSV if not exists
if not os.path.exists(output_file):
    with open(output_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(columns)

In [6]:
async def scrape_combined(url, writer):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        try:
            await page.goto(url, timeout=60000)
            await page.wait_for_load_state('networkidle')
            print(f" Loaded: {url}")

            # Scheme Name
            scheme_name = await page.title()

            # State or Ministry
            state_or_ministry = await page.locator('h3.text-raven').first.text_content()
            state_or_ministry = state_or_ministry.strip() if state_or_ministry else "N/A"
            state = "N/A"
            ministry = "N/A"
            if state_or_ministry in state_names:
                state = state_or_ministry
            elif state_or_ministry in ministry_names:
                ministry = state_or_ministry
            else:
                ministry = state_or_ministry

            # Details
            details = await page.locator('a[href="#details"] ~ div').text_content()
            details = html.unescape(details) if details else "N/A"
            details = unidecode(details).replace("â€œ", "“").replace("â€", "”")

            # Benefits
            benefits = await page.locator('a[href="#benefits"] ~ div').text_content()
            benefits = html.unescape(benefits) if benefits else "N/A"
            benefits = unidecode(benefits).replace("â€œ", "“").replace("â€", "”")

            # Eligibility
            eligibility = await page.locator('a[href="#eligibility"] ~ div').text_content()
            eligibility = eligibility.strip() if eligibility else "N/A"

            # Application Process
            app_process = await page.locator('a[href="#application-process"] ~ div').text_content()
            app_process = app_process.strip() if app_process else "N/A"

            # Documents Required
            documents = await page.locator('a[href="#documents-required"] ~ div').text_content()
            documents = documents.strip() if documents else "N/A"

            # Search Keywords
            keyword_tag = await page.query_selector('meta[name="keywords"]')
            keywords = await keyword_tag.get_attribute('content') if keyword_tag else "N/A"

            # Features
            features_section = await page.query_selector('.scheme-features')
            features = await features_section.inner_text() if features_section else "N/A"

            # Write row
            writer.writerow([
                scheme_name, url, state, ministry, details, benefits, eligibility,
                app_process, documents, keywords, features
            ])
            print(f" Scraped: {scheme_name}")
        except Exception as e:
            print(f" Error scraping {url}: {e}")
        finally:
            await browser.close()

async def scrape_all_combined():
    with open('myscheme_pages_click_test.csv', 'r', encoding='utf-8') as infile:
        reader = csv.reader(infile)
        next(reader)
        urls = [row[1] for row in reader]

    with open(output_file, 'a', newline='', encoding='utf-8') as outfile:
        writer = csv.writer(outfile)
        for url in tqdm(urls, desc="Scraping All Scheme Data", ncols=100):
            await scrape_combined(url, writer)
            await asyncio.sleep(1)

# Run 
await scrape_all_combined()


Scraping All Scheme Data:   0%|                                            | 0/3702 [00:00<?, ?it/s]

 Loaded: https://www.myscheme.gov.in/schemes/fadsp1012e
 Scraped: Financial Assistance To Disabled Students Pursuing (10th, 11th, 12th Equivalent Exams)


Scraping All Scheme Data:   0%|                                 | 1/3702 [00:20<20:38:32, 20.08s/it]

 Loaded: https://www.myscheme.gov.in/schemes/icmr-pdf
 Scraped: ICMR- Post Doctoral Fellowship


Scraping All Scheme Data:   0%|                                 | 2/3702 [00:27<12:45:06, 12.41s/it]

...

Scraping All Scheme Data: 100%|█████████████████████████████▉| 3700/3702 [10:25:46<00:13,  6.88s/it]

 Loaded: https://www.myscheme.gov.in/schemes/majeajcay
 Scraped: Mukhyamantri Anusuchit Jati Evam Anusuchit Janjati Chhatrawas Anudan Yojana


Scraping All Scheme Data: 100%|█████████████████████████████▉| 3701/3702 [10:25:53<00:06,  6.82s/it]

 Loaded: https://www.myscheme.gov.in/schemes/lfvcs
 Scraped: PMMSY: Live Fish Vending Centres Scheme - Haryana


Scraping All Scheme Data: 100%|██████████████████████████████| 3702/3702 [10:25:59<00:00, 10.15s/it]


## Identify the missing URLs and re-scrape only those

In [7]:
with open('myscheme_pages_click_test.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)  # Skip header
    all_urls = {row[1] for row in reader}


In [9]:
with open('scheme_data.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)  # Skip header
    scraped_urls = {row[1] for row in reader}

In [10]:
missing_urls = all_urls - scraped_urls
print(f"{len(missing_urls)} schemes failed and need to be retried.")

338 schemes failed and need to be retried.


In [11]:
with open("missing_urls.csv", "w", newline='', encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["URL"])
    for url in missing_urls:
        writer.writerow([url])

In [1]:
output_file = "scraped_data.csv"         # store final results
failed_file = "missing_urls.csv"         # the list of missing schemes
retry_file = "failed_urls_retry.csv"     # temporary retry file

## Round 1 of retrying failed schemes from missing_urls.csv

In [15]:
output_file = "scraped_data.csv"
missing_file = "missing_urls.csv"
retry_file = "failed_urls_retry.csv"
state_names = {
    'Andhra Pradesh', 'Arunachal Pradesh', 'Assam', 'Bihar', 'Chhattisgarh', 'Goa', 'Gujarat', 'Haryana', 
    'Himachal Pradesh', 'Jharkhand', 'Karnataka', 'Kerala', 'Madhya Pradesh', 'Maharashtra', 'Manipur', 
    'Meghalaya', 'Mizoram', 'Nagaland', 'Odisha', 'Punjab', 'Rajasthan', 'Sikkim', 'Tamil Nadu', 'Telangana', 
    'Tripura', 'Uttar Pradesh', 'Uttarakhand', 'West Bengal', 'Andaman and Nicobar Islands', 'Chandigarh', 
    'Dadra and Nagar Haveli', 'Daman and Diu', 'Lakshadweep', 'Delhi', 'Puducherry', 'Dadra & Nagar Haveli and Daman & Diu',
    'Jammu and Kashmir', 'Ladakh'
    }
ministry_names = {
    'Ministry of Agriculture and Farmers Welfare', 'Ministry of Commerce and Industry', 'Ministry of Defence',
    'Ministry of Education', 'Ministry of Environment, Forest and Climate Change', 'Ministry of Finance',
    'Ministry of Health and Family Welfare', 'Ministry of Home Affairs', 'Ministry of Housing and Urban Affairs',
    'Ministry of Information and Broadcasting', 'Ministry of Labour and Employment', 'Ministry of Law and Justice',
    'Ministry of Micro, Small and Medium Enterprises', 'Ministry of Ministry of Personnel, Public Grievances and Pensions',
    'Ministry of Power', 'Ministry of Railways', 'Ministry of Road Transport and Highways',
    'Ministry of Rural Development', 'Ministry of Science and Technology', 'Ministry of Social Justice and Empowerment',
    'Ministry of Statistics and Programme Implementation', 'Ministry of Women and Child Development',
    'Ministry of Youth Affairs and Sports', 'Ministry of External Affairs', 'Ministry of Tribal Affairs',
    'Ministry of Panchayati Raj', 'Ministry of Skill Development and Entrepreneurship', 'Ministry of Civil Aviation',
    'Ministry of Petroleum and Natural Gas', 'Ministry of Steel', 'Ministry of Labour and Employment'
    

In [17]:
async def scrape_combined(url, writer, failed_writer):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        try:
            await page.goto(url, timeout=60000)
            await page.wait_for_load_state('networkidle')
            print(f" Loaded: {url}")

            scheme_name = await page.title()
            state_or_ministry = await page.locator('h3.text-raven').first.text_content()
            state_or_ministry = state_or_ministry.strip() if state_or_ministry else "N/A"
            state = "N/A"
            ministry = "N/A"
            if state_or_ministry in state_names:
                state = state_or_ministry
            elif state_or_ministry in ministry_names:
                ministry = state_or_ministry
            else:
                ministry = state_or_ministry

            details = await page.locator('a[href="#details"] ~ div').text_content()
            details = html.unescape(details) if details else "N/A"
            details = unidecode(details)

            benefits = await page.locator('a[href="#benefits"] ~ div').text_content()
            benefits = html.unescape(benefits) if benefits else "N/A"
            benefits = unidecode(benefits)

            eligibility = await page.locator('a[href="#eligibility"] ~ div').text_content()
            eligibility = eligibility.strip() if eligibility else "N/A"

            app_process = await page.locator('a[href="#application-process"] ~ div').text_content()
            app_process = app_process.strip() if app_process else "N/A"

            documents = await page.locator('a[href="#documents-required"] ~ div').text_content()
            documents = documents.strip() if documents else "N/A"

            keyword_tag = await page.query_selector('meta[name="keywords"]')
            keywords = await keyword_tag.get_attribute('content') if keyword_tag else "N/A"

            features_section = await page.query_selector('.scheme-features')
            features = await features_section.inner_text() if features_section else "N/A"

            writer.writerow([
                scheme_name, url, state, ministry, details, benefits, eligibility,
                app_process, documents, keywords, features
            ])
            print(f" Scraped: {scheme_name}")
        except Exception as e:
            print(f" Error scraping {url}: {e}")
            failed_writer.writerow([url])
        finally:
            await browser.close()

async def scrape_missing_urls():
    with open(missing_file, 'r', encoding='utf-8') as infile:
        reader = csv.reader(infile)
        next(reader)
        urls = [row[0] for row in reader]

    with open(output_file, 'a', newline='', encoding='utf-8') as outfile, \
         open(retry_file, 'w', newline='', encoding='utf-8') as failedfile:

        writer = csv.writer(outfile)
        failed_writer = csv.writer(failedfile)
        failed_writer.writerow(["URL"])

        for url in tqdm(urls, desc="Retrying Missing URLs", ncols=100):
            await scrape_combined(url, writer, failed_writer)
            await asyncio.sleep(1)

# Run 
await scrape_missing_urls()


Retrying Missing URLs:   0%|                                                | 0/338 [00:00<?, ?it/s]

 Loaded: https://www.myscheme.gov.in/schemes/pmgdisha
 Scraped: Pradhan Mantri Gramin Digital Saksharta Abhiyaan


Retrying Missing URLs:   0%|                                      | 1/338 [00:16<1:35:26, 16.99s/it]

 Loaded: https://www.myscheme.gov.in/schemes/ahstc-dc
 Scraped: "Advance/High Skill Training in Coir" Component of the "Development of Coir" Scheme


Retrying Missing URLs:   1%|▏                                     | 2/338 [00:23<1:01:11, 10.93s/it]

...

Retrying Missing URLs:  99%|█████████████████████████████████████▊| 336/338 [56:47<00:42, 21.22s/it]

 Loaded: https://www.myscheme.gov.in/schemes/sss-gujarat
 Scraped: Sant Surdas Scheme


Retrying Missing URLs: 100%|█████████████████████████████████████▉| 337/338 [56:55<00:17, 17.11s/it]

 Loaded: https://www.myscheme.gov.in/schemes/rosftcw
 Scraped: Refund of Subscriptions for the Construction Workers


Retrying Missing URLs: 100%|██████████████████████████████████████| 338/338 [57:02<00:00, 10.13s/it]


## Retry 2

In [38]:
with open('myscheme_pages_click_test.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)  # Skip header
    all_urls = {row[1] for row in reader}

In [39]:
with open('scraped_data.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)  # Skip header
    scraped_urls = {row[1] for row in reader}

In [40]:
with open('scheme_data.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)  # Skip header
    scheme_urls = {row[1] for row in reader}

In [41]:
missing_urls = all_urls - scraped_urls - scheme_urls
print(f"{len(missing_urls)} schemes failed and need to be retried.")

34 schemes failed and need to be retried.


In [42]:
with open("missing_urls_2.csv", "w", newline='', encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["URL"])
    for url in missing_urls:
        writer.writerow([url])

### Main scraping function (with timeout handling and logging failed URLs)

In [60]:
# Files
missing_file = "missing_urls_2.csv"   
output_file = "scraped_data_2.csv"  
state_names = {
    'Andhra Pradesh', 'Arunachal Pradesh', 'Assam', 'Bihar', 'Chhattisgarh', 'Goa', 'Gujarat', 'Haryana', 
    'Himachal Pradesh', 'Jharkhand', 'Karnataka', 'Kerala', 'Madhya Pradesh', 'Maharashtra', 'Manipur', 
    'Meghalaya', 'Mizoram', 'Nagaland', 'Odisha', 'Punjab', 'Rajasthan', 'Sikkim', 'Tamil Nadu', 'Telangana', 
    'Tripura', 'Uttar Pradesh', 'Uttarakhand', 'West Bengal', 'Andaman and Nicobar Islands', 'Chandigarh', 
    'Dadra and Nagar Haveli', 'Daman and Diu', 'Lakshadweep', 'Delhi', 'Puducherry', 'Dadra & Nagar Haveli and Daman & Diu',
    'Jammu and Kashmir', 'Ladakh'
    }
ministry_names = {
    'Ministry of Agriculture and Farmers Welfare', 'Ministry of Commerce and Industry', 'Ministry of Defence',
    'Ministry of Education', 'Ministry of Environment, Forest and Climate Change', 'Ministry of Finance',
    'Ministry of Health and Family Welfare', 'Ministry of Home Affairs', 'Ministry of Housing and Urban Affairs',
    'Ministry of Information and Broadcasting', 'Ministry of Labour and Employment', 'Ministry of Law and Justice',
    'Ministry of Micro, Small and Medium Enterprises', 'Ministry of Ministry of Personnel, Public Grievances and Pensions',
    'Ministry of Power', 'Ministry of Railways', 'Ministry of Road Transport and Highways',
    'Ministry of Rural Development', 'Ministry of Science and Technology', 'Ministry of Social Justice and Empowerment',
    'Ministry of Statistics and Programme Implementation', 'Ministry of Women and Child Development',
    'Ministry of Youth Affairs and Sports', 'Ministry of External Affairs', 'Ministry of Tribal Affairs',
    'Ministry of Panchayati Raj', 'Ministry of Skill Development and Entrepreneurship', 'Ministry of Civil Aviation',
    'Ministry of Petroleum and Natural Gas', 'Ministry of Steel', 'Ministry of Labour and Employment'
    }

async def scrape_combined(url, writer):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        try:
            await page.goto(url, timeout=60000)
            await page.wait_for_load_state('networkidle')
            print(f" Loaded: {url}")

            scheme_name = await page.title()

            state_or_ministry = await page.locator('h3.text-raven').first.text_content()
            state_or_ministry = state_or_ministry.strip() if state_or_ministry else "N/A"
            state, ministry = "N/A", "N/A"
            if state_or_ministry in state_names:
                state = state_or_ministry
            elif state_or_ministry in ministry_names:
                ministry = state_or_ministry
            else:
                ministry = state_or_ministry

            details = await page.locator('a[href="#details"] ~ div').text_content() or "N/A"
            benefits = await page.locator('a[href="#benefits"] ~ div').text_content() or "N/A"
            eligibility = await page.locator('a[href="#eligibility"] ~ div').text_content() or "N/A"
            app_process = await page.locator('a[href="#application-process"] ~ div').text_content() or "N/A"
            documents = await page.locator('a[href="#documents-required"] ~ div').text_content() or "N/A"

            keyword_tag = await page.query_selector('meta[name="keywords"]')
            keywords = await keyword_tag.get_attribute('content') if keyword_tag else "N/A"

            features_section = await page.query_selector('.scheme-features')
            features = await features_section.inner_text() if features_section else "N/A"

            writer.writerow([
                scheme_name, url, state, ministry,
                unidecode(html.unescape(details)),
                unidecode(html.unescape(benefits)),
                eligibility.strip(),
                app_process.strip(),
                documents.strip(),
                keywords,
                features
            ])
            print(f"Scraped: {scheme_name}")

        except Exception as e:
            print(f" Error scraping {url}: {e}")
        finally:
            await browser.close()



In [61]:
async def scrape_missing_urls():
    with open(missing_file, 'r', encoding='utf-8') as infile:
        reader = csv.reader(infile)
        next(reader)  # Skip header
        urls = [row[0] for row in reader]

    with open(output_file, 'a', newline='', encoding='utf-8') as outfile:
        writer = csv.writer(outfile)

        # Optional: write header if file is empty
        if outfile.tell() == 0:
            writer.writerow([
                "Scheme Name", "URL", "State", "Ministry", "Details", "Benefits", "Eligibility",
                "Application Process", "Documents Required", "Search Keywords", "Features"
            ])

        for url in tqdm(urls, desc="Scraping Remaining Schemes", ncols=100):
            await scrape_combined(url, writer)
            await asyncio.sleep(1)


In [62]:
await scrape_missing_urls()

Scraping Remaining Schemes:   0%|                                            | 0/34 [00:00<?, ?it/s]

 Loaded: https://www.myscheme.gov.in/schemes/els-cl2mc
Scraped: Education Loan Scheme- Credit Line 2 for Minority Community


Scraping Remaining Schemes:   3%|█                                   | 1/34 [00:08<04:54,  8.93s/it]

 Loaded: https://www.myscheme.gov.in/schemes/agsmais
Scraped: Aatmanirbhar Gujarat Scheme for Assistance to Mega Industries: Assistance of Interest Subsidy


Scraping Remaining Schemes:   6%|██                                  | 2/34 [00:16<04:17,  8.04s/it]

...

Scraping Remaining Schemes:  94%|████████████████████████████████▉  | 32/34 [03:48<00:13,  6.94s/it]

 Loaded: https://www.myscheme.gov.in/schemes/mjpjay
Scraped: Mahatma Jyotirao Phule Jan Arogya Yojana


Scraping Remaining Schemes:  97%|█████████████████████████████████▉ | 33/34 [03:55<00:06,  6.96s/it]

 Loaded: https://www.myscheme.gov.in/schemes/assy-goa
 Error scraping https://www.myscheme.gov.in/schemes/assy-goa: Locator.text_content: Timeout 30000ms exceeded.
Call log:
  - waiting for locator("h3.text-raven").first



Scraping Remaining Schemes: 100%|███████████████████████████████████| 34/34 [04:32<00:00,  8.00s/it]


## Retry 3

In [65]:
with open("missing_urls_3.csv", "w", newline='', encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["URL"])
    writer.writerow(["https://www.myscheme.gov.in/schemes/assy-goa"])


In [66]:
# Files
missing_file = "missing_urls_3.csv"   
output_file = "scraped_data_3.csv"  

In [67]:
await scrape_missing_urls()

Scraping Remaining Schemes:   0%|                                             | 0/1 [00:00<?, ?it/s]

 Loaded: https://www.myscheme.gov.in/schemes/assy-goa
Scraped: Antya Sanskar Sahay Yojana


Scraping Remaining Schemes: 100%|█████████████████████████████████████| 1/1 [00:07<00:00,  7.12s/it]


## Combine all the 4 files

In [68]:
import pandas as pd
import glob

In [75]:
csv_files = [
    "scheme_data.csv",
    "scraped_data.csv",
    "scraped_data_2.csv",
    "scraped_data_3.csv"
    ]

In [76]:
dfs = [pd.read_csv(file) for file in csv_files]
combined_df = pd.concat(dfs, ignore_index=True)

In [81]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3702 entries, 0 to 3701
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Scheme Name          3702 non-null   object 
 1   URL                  3702 non-null   object 
 2   State                3121 non-null   object 
 3   Ministry             581 non-null    object 
 4   Details              3702 non-null   object 
 5   Benefits             3702 non-null   object 
 6   Eligibility          3702 non-null   object 
 7   Application Process  3702 non-null   object 
 8   Documents Required   3701 non-null   object 
 9   Search Keywords      3702 non-null   object 
 10  Features             0 non-null      float64
dtypes: float64(1), object(10)
memory usage: 318.3+ KB


### Handle duplicates

In [83]:
combined_df.duplicated().sum()

1

In [84]:
combined_df[combined_df.duplicated(subset=["URL"], keep=False)]

,Scheme Name,URL,State,Ministry,Details,Benefits,Eligibility,Application Process,Documents Required,Search Keywords,Features
1330,Family Pension (BBOCWWB),https://www.myscheme.gov.in/schemes/fpbbocwwb,Bihar,NaN,"The scheme ""Family Pension"" by the Bihar Build...","50% of the pension amount or Rs100/-, whicheve...",The applicant should be a dependent of a decea...,OnlineRegistration of a Construction Worker un...,"Identity Proof i.e. Aadhaar Card, Voter CardBa...","Family Pension (BBOCWWB), fpbbocwwb, Building,...",NaN
2668,Family Pension (BBOCWWB),https://www.myscheme.gov.in/schemes/fpbbocwwb,Bihar,NaN,"The scheme ""Family Pension"" by the Bihar Build...","50% of the pension amount or Rs100/-, whicheve...",The applicant should be a dependent of a decea...,OnlineRegistration of a Construction Worker un...,"Identity Proof i.e. Aadhaar Card, Voter CardBa...","Family Pension (BBOCWWB), fpbbocwwb, Building,...",NaN


In [85]:
# Drop duplicates 
combined_df.drop_duplicates(subset=["URL"], inplace=True)

In [86]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3701 entries, 0 to 3701
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Scheme Name          3701 non-null   object 
 1   URL                  3701 non-null   object 
 2   State                3120 non-null   object 
 3   Ministry             581 non-null    object 
 4   Details              3701 non-null   object 
 5   Benefits             3701 non-null   object 
 6   Eligibility          3701 non-null   object 
 7   Application Process  3701 non-null   object 
 8   Documents Required   3700 non-null   object 
 9   Search Keywords      3701 non-null   object 
 10  Features             0 non-null      float64
dtypes: float64(1), object(10)
memory usage: 347.0+ KB


In [87]:
# Locate the Row with Missing Documents
missing_doc = combined_df[combined_df["Documents Required"].isna()]
missing_doc[["Scheme Name", "URL"]]

,Scheme Name,URL
2786,Pradhan Mantri Suraksha Bima Yojana,https://www.myscheme.gov.in/schemes/pmsby


In [89]:
missing_url = missing_doc.iloc[0]["URL"]
print(missing_url)

https://www.myscheme.gov.in/schemes/pmsby


### Rescrape for 1 scheme

In [91]:
with open("missing_urls_4.csv", "w", newline='', encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["URL"])
    writer.writerow(["https://www.myscheme.gov.in/schemes/pmsby"])

In [92]:
# Files
missing_file = "missing_urls_4.csv"   
output_file = "scraped_data_4.csv"  

In [93]:
await scrape_missing_urls()

Scraping Remaining Schemes:   0%|                                             | 0/1 [00:00<?, ?it/s]

 Loaded: https://www.myscheme.gov.in/schemes/pmsby
Scraped: Pradhan Mantri Suraksha Bima Yojana


Scraping Remaining Schemes: 100%|█████████████████████████████████████| 1/1 [00:08<00:00,  8.41s/it]


In [94]:
one_schme = pd.read_csv("scraped_data_4.csv")

In [95]:
one_schme.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Scheme Name          1 non-null      object 
 1   URL                  1 non-null      object 
 2   State                0 non-null      float64
 3   Ministry             1 non-null      object 
 4   Details              1 non-null      object 
 5   Benefits             1 non-null      object 
 6   Eligibility          1 non-null      object 
 7   Application Process  1 non-null      object 
 8   Documents Required   0 non-null      float64
 9   Search Keywords      1 non-null      object 
 10  Features             0 non-null      float64
dtypes: float64(3), object(8)
memory usage: 220.0+ bytes


### Note:
"Documents Required" section is truly not present on the scheme's webpage

### Fill missing values meaningfully:

In [97]:
combined_df["Documents Required"].fillna("Not Mentioned", inplace=True)
combined_df["Features"].fillna("Not Available", inplace=True)
combined_df["State"].fillna("N/A", inplace=True)

In [99]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3701 entries, 0 to 3701
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Scheme Name          3701 non-null   object
 1   URL                  3701 non-null   object
 2   State                3701 non-null   object
 3   Ministry             581 non-null    object
 4   Details              3701 non-null   object
 5   Benefits             3701 non-null   object
 6   Eligibility          3701 non-null   object
 7   Application Process  3701 non-null   object
 8   Documents Required   3701 non-null   object
 9   Search Keywords      3701 non-null   object
 10  Features             3701 non-null   object
dtypes: object(11)
memory usage: 347.0+ KB


In [101]:
combined_df.to_csv("final_myscheme_dataset.csv", index=False)